<a href="https://colab.research.google.com/github/soumik-aj12/LaLiga-Match-Prediction/blob/main/LaLiga_Match_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

We are using FBREF website to scrape the data.

In [ ]:
url = "https://fbref.com/en/comps/12/La-Liga-Stats"

# Creating a list named 'years' to store the years we want to scrape data for.
We use range() to generate a sequence of numbers from 2022 down to 2020 (exclusive of 2019),  with a step of -1 (decreasing order).
Finally, we convert this sequence to a list using list().

In [ ]:
years = list(range(2022,2019,-1))
years

[2024, 2023, 2022, 2021, 2020]

# Initialize an empty list called 'all_matches'.
This list will store all the match data we scrape for each team and season.

# Define the URL for the initial standings page of La Liga.
We will use this URL as a starting point to navigate to other pages and scrape data.

In [ ]:
all_matches = []
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

#Iterate through each year in the 'years' list and Fetch the HTML content of the standings page
Then iterate through the team's URL

In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        matches = pd.read_html(data.text,match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()               #dropping multilevel columns
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(15)

<ipython-input-5-61aff408fbd1>:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
<ipython-input-5-61aff408fbd1>:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text,match="Scores & Fixtures")[0]
<ipython-input-5-61aff408fbd1>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
<ipython-input-5-61aff408fbd1>:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' obj

In [ ]:
all_matches[0]

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,Valencia,...,Match Report,NaN,17,5,18.6,1,1,1,2024,Barcelona
1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,Athletic Club,...,Match Report,NaN,13,5,16.6,0,0,0,2024,Barcelona
2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,Rayo Vallecano,...,Match Report,NaN,22,5,19.3,1,0,0,2024,Barcelona
3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,Valladolid,...,Match Report,NaN,23,11,13.7,1,0,0,2024,Barcelona
4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,Girona,...,Match Report,NaN,20,9,19.1,0,0,0,2024,Barcelona
6,2024-09-22,18:30,La Liga,Matchweek 6,Sun,Away,W,5.0,1.0,Villarreal,...,Match Report,NaN,16,9,14.6,0,0,1,2024,Barcelona
7,2024-09-25,21:00,La Liga,Matchweek 7,Wed,Home,W,1.0,0.0,Getafe,...,Match Report,NaN,15,4,19.1,2,0,0,2024,Barcelona
8,2024-09-28,21:00,La Liga,Matchweek 8,Sat,Away,L,2.0,4.0,Osasuna,...,Match Report,NaN,12,6,19.8,1,0,0,2024,Barcelona
10,2024-10-06,16:15,La Liga,Matchweek 9,Sun,Away,W,3.0,0.0,Alavés,...,Match Report,NaN,15,9,16.8,1,0,0,2024,Barcelona
11,2024-10-20,21:00,La Liga,Matchweek 10,Sun,Home,W,5.0,1.0,Sevilla,...,Match Report,NaN,20,8,17.9,1,1,1,2024,Barcelona


In [ ]:
len(all_matches)

100

In [ ]:
matches_df = pd.concat(all_matches)

In [ ]:
matches_df.columns = [c.lower() for c in matches_df.columns]
matches_df.columns

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'opp formation', 'referee', 'match report', 'notes', 'sh', 'sot',
       'dist', 'fk', 'pk', 'pkatt', 'season', 'team'],
      dtype='object')

In [ ]:
matches_df.to_csv("matches.csv")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/LaLiga_Dataset/matches.csv')
data.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,...,Match Report,NaN,17.0,5.0,18.6,1.0,1,1,2023,Barcelona
1,1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,...,Match Report,NaN,13.0,5.0,16.6,0.0,0,0,2023,Barcelona
2,2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,...,Match Report,NaN,22.0,5.0,19.3,1.0,0,0,2023,Barcelona
3,3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,...,Match Report,NaN,23.0,11.0,13.7,1.0,0,0,2023,Barcelona
4,4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,...,Match Report,NaN,20.0,9.0,19.1,0.0,0,0,2023,Barcelona


In [ ]:
38*20*5

3800

In [ ]:
data["date"] = pd.to_datetime(data["date"])

In [ ]:
data["venue_code"] = data["venue"].astype("category").cat.codes

In [ ]:
data["opponent_code"] = data["opponent"].astype("category").cat.codes

In [ ]:
data["hour"] = data["time"].str.replace(":.+", "", regex=True).astype("int")

In [ ]:
data["day_code"] = data["date"].dt.dayofweek

In [ ]:
data["target"] = (data["result"] == "W").astype("int")

In [ ]:
data.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,fk,pk,pkatt,season,team,venue_code,opponent_code,hour,day_code,target
0,0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,...,1.0,1,1,2023,Barcelona,0,22,21,5,1
1,1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,...,0.0,0,0,2023,Barcelona,1,2,19,5,1
2,2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,...,1.0,0,0,2023,Barcelona,0,18,21,1,1
3,3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,...,1.0,0,0,2023,Barcelona,1,23,17,5,1
4,4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,...,0.0,0,0,2023,Barcelona,0,11,16,6,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [ ]:
rf = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=1)

In [ ]:
train = data[data["date"] < '2022-01-01']

In [ ]:
test = data[data["date"] > '2022-01-01']

In [ ]:
pred = ["venue_code", "opponent_code", "hour", "day_code"]

In [ ]:
rf.fit(train[pred], train["target"])

RandomForestClassifier(min_samples_split=10, random_state=1)

In [ ]:
prediction = rf.predict(test[pred])

In [ ]:
accuracy = accuracy_score(test["target"], prediction)

In [ ]:
accuracy

0.627205199628598

In [ ]:
combine = pd.DataFrame(dict(actual=test["target"], predicted=prediction))

In [ ]:
pd.crosstab(index=combine["actual"],columns=combine["predicted"])

predicted,0,1
actual,,
0,1158,198
1,605,193


In [ ]:
precision_score(test["target"], prediction)

0.4936061381074169

In [ ]:
grouped_matches = data.groupby("team")

In [ ]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("date")
  rolling_stats = group[cols].rolling(3, closed='left').mean()
  group[new_cols] = rolling_stats
  group = group.dropna(subset=new_cols)
  return group

In [ ]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
matches_rolling = data.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

<ipython-input-27-22258457637b>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = data.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [ ]:
matches_rolling

Unnamed: 0       date   time     comp         round  day  \
team                                                                        
Alaves     2483           3 2021-09-18  21:00  La Liga   Matchweek 5  Sat   
           2484           4 2021-09-22  19:30  La Liga   Matchweek 6  Wed   
           2485           5 2021-09-25  14:00  La Liga   Matchweek 7  Sat   
           2486           6 2021-10-01  21:00  La Liga   Matchweek 8  Fri   
           2487           7 2021-10-18  19:00  La Liga   Matchweek 9  Mon   
...                     ...        ...    ...      ...           ...  ...   
Villarreal 41             6 2024-09-26  19:00  La Liga   Matchweek 7  Thu   
           42             7 2024-09-30  21:00  La Liga   Matchweek 8  Mon   
           43             8 2024-10-05  21:00  La Liga   Matchweek 9  Sat   
           44             9 2024-10-20  18:30  La Liga  Matchweek 10  Sun   
           45            10 2024-10-26  14:00  La Liga  Matchweek 11  Sat   

                venue result   gf   ga  ... day_code  target  gf_rolling  \
team                                    ...                                
Alaves     2483  Home      L  0.0  2.0  ...        5       0    0.333333   
           2484  Away      L  0.0  1.0  ...        2       0    0.000000   
           2485  Home      W  1.0  0.0  ...        5       1    0.000000   
           2486  Away      L  0.0  1.0  ...        4       0    0.333333   
           2487  Home      L  0.0  1.0  ...        0       0    0.333333   
...               ...    ...  ...  ...  ...      ...     ...         ...   
Villarreal 41    Away      W  2.0  1.0  ...        3       1    1.333333   
           42    Home      W  3.0  1.0  ...        0       1    1.666667   
           43    Away      L  0.0  2.0  ...        5       0    2.000000   
           44    Home      D  1.0  1.0  ...        6       0    1.666667   
           45    Away      W  2.0  1.0  ...        5       1    1.333333   

                 ga_rolling  sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                          
Alaves     2483    2.666667    9.333333    3.000000    17.766667   0.666667   
           2484    2.000000    8.666667    3.000000    18.866667   0.333333   
           2485    2.000000    8.000000    2.333333    18.900000   0.333333   
           2486    1.000000    7.333333    2.000000    17.833333   0.000000   
           2487    0.666667   10.333333    2.666667    15.100000   0.000000   
...                     ...         ...         ...          ...        ...   
Villarreal 41      2.333333   14.333333    3.333333    15.733333   0.000000   
           42      2.333333   16.333333    4.000000    15.933333   0.000000   
           43      2.333333   15.666667    4.666667    15.600000   0.000000   
           44      1.333333   15.333333    3.666667    16.466667   0.333333   
           45      1.333333   13.666667    3.666667    16.233333   0.333333   

                pk_rolling pkatt_rolling  
team                                      
Alaves     2483   0.333333      0.333333  
           2484   0.000000      0.000000  
           2485   0.000000      0.000000  
           2486   0.000000      0.000000  
           2487   0.000000      0.000000  
...                    ...           ...  
Villarreal 41     0.000000      0.000000  
           42     0.000000      0.000000  
           43     0.000000      0.333333  
           44     0.000000      0.333333  
           45     0.000000      0.333333  

[2440 rows x 42 columns]

In [ ]:
matches_rolling = matches_rolling.droplevel('team')

In [ ]:
matches_rolling

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
2483,3,2021-09-18,21:00,La Liga,Matchweek 5,Sat,Home,L,0.0,2.0,...,5,0,0.333333,2.666667,9.333333,3.000000,17.766667,0.666667,0.333333,0.333333
2484,4,2021-09-22,19:30,La Liga,Matchweek 6,Wed,Away,L,0.0,1.0,...,2,0,0.000000,2.000000,8.666667,3.000000,18.866667,0.333333,0.000000,0.000000
2485,5,2021-09-25,14:00,La Liga,Matchweek 7,Sat,Home,W,1.0,0.0,...,5,1,0.000000,2.000000,8.000000,2.333333,18.900000,0.333333,0.000000,0.000000
2486,6,2021-10-01,21:00,La Liga,Matchweek 8,Fri,Away,L,0.0,1.0,...,4,0,0.333333,1.000000,7.333333,2.000000,17.833333,0.000000,0.000000,0.000000
2487,7,2021-10-18,19:00,La Liga,Matchweek 9,Mon,Home,L,0.0,1.0,...,0,0,0.333333,0.666667,10.333333,2.666667,15.100000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,6,2024-09-26,19:00,La Liga,Matchweek 7,Thu,Away,W,2.0,1.0,...,3,1,1.333333,2.333333,14.333333,3.333333,15.733333,0.000000,0.000000,0.000000
42,7,2024-09-30,21:00,La Liga,Matchweek 8,Mon,Home,W,3.0,1.0,...,0,1,1.666667,2.333333,16.333333,4.000000,15.933333,0.000000,0.000000,0.000000
43,8,2024-10-05,21:00,La Liga,Matchweek 9,Sat,Away,L,0.0,2.0,...,5,0,2.000000,2.333333,15.666667,4.666667,15.600000,0.000000,0.000000,0.333333
44,9,2024-10-20,18:30,La Liga,Matchweek 10,Sun,Home,D,1.0,1.0,...,6,0,1.666667,1.333333,15.333333,3.666667,16.466667,0.333333,0.000000,0.333333


In [ ]:
matches_rolling.index = range(matches_rolling.shape[0])

In [ ]:
matches_rolling.drop(["Unnamed: 0"], axis = 1)

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,2021-09-18,21:00,La Liga,Matchweek 5,Sat,Home,L,0.0,2.0,Osasuna,...,5,0,0.333333,2.666667,9.333333,3.000000,17.766667,0.666667,0.333333,0.333333
1,2021-09-22,19:30,La Liga,Matchweek 6,Wed,Away,L,0.0,1.0,Espanyol,...,2,0,0.000000,2.000000,8.666667,3.000000,18.866667,0.333333,0.000000,0.000000
2,2021-09-25,14:00,La Liga,Matchweek 7,Sat,Home,W,1.0,0.0,Atlético Madrid,...,5,1,0.000000,2.000000,8.000000,2.333333,18.900000,0.333333,0.000000,0.000000
3,2021-10-01,21:00,La Liga,Matchweek 8,Fri,Away,L,0.0,1.0,Athletic Club,...,4,0,0.333333,1.000000,7.333333,2.000000,17.833333,0.000000,0.000000,0.000000
4,2021-10-18,19:00,La Liga,Matchweek 9,Mon,Home,L,0.0,1.0,Betis,...,0,0,0.333333,0.666667,10.333333,2.666667,15.100000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,2024-09-26,19:00,La Liga,Matchweek 7,Thu,Away,W,2.0,1.0,Espanyol,...,3,1,1.333333,2.333333,14.333333,3.333333,15.733333,0.000000,0.000000,0.000000
2436,2024-09-30,21:00,La Liga,Matchweek 8,Mon,Home,W,3.0,1.0,Las Palmas,...,0,1,1.666667,2.333333,16.333333,4.000000,15.933333,0.000000,0.000000,0.000000
2437,2024-10-05,21:00,La Liga,Matchweek 9,Sat,Away,L,0.0,2.0,Real Madrid,...,5,0,2.000000,2.333333,15.666667,4.666667,15.600000,0.000000,0.000000,0.333333
2438,2024-10-20,18:30,La Liga,Matchweek 10,Sun,Home,D,1.0,1.0,Getafe,...,6,0,1.666667,1.333333,15.333333,3.666667,16.466667,0.333333,0.000000,0.333333


In [ ]:
def make_predictions(data, predictors):
  train = data[data["date"] < '2022-01-01']
  test = data[data["date"] > '2022-01-01']
  rf.fit(train[pred], train["target"])
  prediction = rf.predict(test[pred])
  accuracy = accuracy_score(test["target"], prediction)
  combined = pd.DataFrame(dict(actual=test["target"], predicted=prediction))
  precision = precision_score(test["target"], prediction)
  return combined, precision

In [ ]:
combined, precision = make_predictions(matches_rolling, pred)

In [ ]:
precision

0.4888268156424581

In [ ]:
result = combined.merge(matches_rolling[["date","team","opponent","result" ]],left_index=True, right_index =True)

In [ ]:
result

,actual,predicted,date,team,opponent,result
15,0,0,2022-01-02,Alaves,Real Sociedad,D
16,0,0,2022-01-09,Alaves,Athletic Club,D
17,0,0,2022-01-18,Alaves,Betis,L
18,0,0,2022-01-23,Alaves,Barcelona,L
19,0,0,2022-02-05,Alaves,Elche,L
...,...,...,...,...,...,...
2435,1,0,2024-09-26,Villarreal,Espanyol,W
2436,1,0,2024-09-30,Villarreal,Las Palmas,W
2437,0,0,2024-10-05,Villarreal,Real Madrid,L
2438,0,0,2024-10-20,Villarreal,Getafe,D


In [ ]:
class MissingDictionary(dict):
  __missing__ = lambda self, key: key
map_values = {
      "Leganés": "Leganes",
      "Alavés": "Alaves",
      "Cádiz": "Cadiz",
      "Almería": "Almeria",
      "Betis": "Real Betis",
      "Atlético Madrid": "Atletico Madrid"
  }
mapping = MissingDictionary(**map_values)

In [ ]:
result["new_team"] = result["team"].map(mapping)

In [ ]:
result

,actual,predicted,date,team,opponent,result,new_team
15,0,0,2022-01-02,Alaves,Real Sociedad,D,Alaves
16,0,0,2022-01-09,Alaves,Athletic Club,D,Alaves
17,0,0,2022-01-18,Alaves,Betis,L,Alaves
18,0,0,2022-01-23,Alaves,Barcelona,L,Alaves
19,0,0,2022-02-05,Alaves,Elche,L,Alaves
...,...,...,...,...,...,...,...
2435,1,0,2024-09-26,Villarreal,Espanyol,W,Villarreal
2436,1,0,2024-09-30,Villarreal,Las Palmas,W,Villarreal
2437,0,0,2024-10-05,Villarreal,Real Madrid,L,Villarreal
2438,0,0,2024-10-20,Villarreal,Getafe,D,Villarreal


In [ ]:
merged = result.merge(result, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [ ]:
merged

,actual_x,predicted_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,predicted_y,team_y,opponent_y,result_y,new_team_y
0,1,0,2022-01-03,Athletic Club,Osasuna,W,Athletic Club,0,0,Osasuna,Athletic Club,L,Osasuna
1,0,0,2022-01-09,Athletic Club,Alavés,D,Athletic Club,0,0,Alaves,Athletic Club,D,Alaves
2,1,0,2022-01-23,Athletic Club,Rayo Vallecano,W,Athletic Club,0,0,Rayo Vallecano,Athletic Club,L,Rayo Vallecano
3,1,0,2022-02-07,Athletic Club,Espanyol,W,Athletic Club,0,0,Espanyol,Athletic Club,L,Espanyol
4,0,0,2022-02-14,Athletic Club,Mallorca,L,Athletic Club,1,0,Mallorca,Athletic Club,W,Mallorca
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1650,1,0,2024-09-26,Villarreal,Espanyol,W,Villarreal,0,0,Espanyol,Villarreal,L,Espanyol
1651,1,0,2024-09-30,Villarreal,Las Palmas,W,Villarreal,0,0,Las Palmas,Villarreal,L,Las Palmas
1652,0,0,2024-10-05,Villarreal,Real Madrid,L,Villarreal,1,0,Real Madrid,Villarreal,W,Real Madrid
1653,0,0,2024-10-20,Villarreal,Getafe,D,Villarreal,0,0,Getafe,Villarreal,D,Getafe


In [ ]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] == 0)]["actual_x"].value_counts()

,count
actual_x,
0,129
1,124


In [ ]:
acc = 129/(129+124)
print(f"{acc*100} %")

50.988142292490124 %
